In [1]:
import numpy as np
import pyarrow as pa
import pyarrow.csv as csv
import pandas as pd

In [26]:
!ls -l /home/wesm/code/pydata-book/datasets/fec/

total 154176
-rw-r--r-- 1 wesm wesm 157871393 Feb 25 12:03 P00000001-ALL.csv


In [27]:
!head -n 5 /home/wesm/code/pydata-book/datasets/fec/P00000001-ALL.csv

cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num
C00410118,"P20002978","Bachmann, Michelle","HARVEY, WILLIAM","MOBILE","AL","366010290","RETIRED","RETIRED",250,20-JUN-11,"","","","SA17A",736166
C00410118,"P20002978","Bachmann, Michelle","HARVEY, WILLIAM","MOBILE","AL","366010290","RETIRED","RETIRED",50,23-JUN-11,"","","","SA17A",736166
C00410118,"P20002978","Bachmann, Michelle","SMITH, LANIER","LANETT","AL","368633403","INFORMATION REQUESTED","INFORMATION REQUESTED",250,05-JUL-11,"","","","SA17A",749073
C00410118,"P20002978","Bachmann, Michelle","BLEVINS, DARONDA","PIGGOTT","AR","724548253","NONE","RETIRED",250,01-AUG-11,"","","","SA17A",749073


In [28]:
fec = pd.read_csv('/home/wesm/code/pydata-book/datasets/fec/P00000001-ALL.csv')
fec.head()
def coerce_int(x):
    try:
        return int(x)
    except:
        return -1

fec['contbr_zip'] = fec['contbr_zip'].map(coerce_int).astype(np.int64)

In [30]:
arrow_fec = pa.Table.from_pandas(fec, preserve_index=False)

In [31]:
arrow_fec.schema

cmte_id: string
cand_id: string
cand_nm: string
contbr_nm: string
contbr_city: string
contbr_st: string
contbr_zip: int64
contbr_employer: string
contbr_occupation: string
contb_receipt_amt: double
contb_receipt_dt: string
receipt_desc: string
memo_cd: string
memo_text: string
form_tp: string
file_num: int64
metadata
--------
{b'pandas': b'{"index_columns": [], "column_indexes": [], "columns": [{"name":'
            b' "cmte_id", "field_name": "cmte_id", "pandas_type": "unicode", "'
            b'numpy_type": "object", "metadata": null}, {"name": "cand_id", "f'
            b'ield_name": "cand_id", "pandas_type": "unicode", "numpy_type": "'
            b'object", "metadata": null}, {"name": "cand_nm", "field_name": "c'
            b'and_nm", "pandas_type": "unicode", "numpy_type": "object", "meta'
            b'data": null}, {"name": "contbr_nm", "field_name": "contbr_nm", "'
            b'pandas_type": "unicode", "numpy_type": "object", "metadata": nul'
            b'l}, {"name": "cont

In [2]:
pa.total_allocated_bytes()

0

In [33]:
with open('fec.arrow', 'wb') as f:
    writer = pa.ipc.RecordBatchFileWriter(f, arrow_fec.schema)
    for i in range(50):
        writer.write(arrow_fec)
    writer.close()

In [34]:
!ls -l

total 8692860
-rw-r--r-- 1 wesm wesm       3593 May 10 13:28 20190225arrow-Copy1.ipynb
-rw-r--r-- 1 wesm wesm     111635 May 10 13:24 20190225arrow.ipynb
-rw-r--r-- 1 wesm wesm 8901358498 May 10 13:56 fec.arrow
-rw-r--r-- 1 wesm wesm        688 Mar  1 10:55 Untitled.ipynb


In [3]:
mmap = pa.memory_map('fec.arrow')
f = pa.ipc.open_file(mmap)

In [4]:
t = f.read_all()

In [5]:
t[0].data.num_chunks

50

In [ ]:
pa.total_allocated_bytes()

In [ ]:
arr = t[0].data.chunk(0)

In [ ]:
arr[0]

In [6]:
%time t[1].unique()

CPU times: user 422 ms, sys: 15.3 ms, total: 438 ms
Wall time: 436 ms


[
  "P20002978",
  "P80003353",
  "P80003338",
  "P20002523",
  "P20002556",
  "P20002671",
  "P80000748",
  "P20002721",
  "P00003608",
  "P60003654",
  "P20003109",
  "P20003067",
  "P20003281"
]